artificial neural networks

libraries

In [11]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler #important to scale your data before training a nn.
from sklearn.linear_model import LinearRegression
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

2025-10-01 14:47:10.977115: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-01 14:47:11.652707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-01 14:47:15.033075: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


data

In [5]:
df = pd.read_csv('hourly_wages.csv')
df.head()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,5.10,0,8,21,35,1,1,0,1,0
1,4.95,0,9,42,57,1,1,0,1,0
2,6.67,0,12,1,19,0,0,0,1,0
3,4.00,0,12,4,22,0,0,0,0,0
4,7.50,0,12,17,35,0,1,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   wage_per_hour   534 non-null    float64
 1   union           534 non-null    int64  
 2   education_yrs   534 non-null    int64  
 3   experience_yrs  534 non-null    int64  
 4   age             534 non-null    int64  
 5   female          534 non-null    int64  
 6   marr            534 non-null    int64  
 7   south           534 non-null    int64  
 8   manufacturing   534 non-null    int64  
 9   construction    534 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 41.8 KB


In [7]:
df.describe()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
count,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000
mean,9.024064,0.179775,13.018727,17.822097,36.833333,0.458801,0.655431,0.292135,0.185393,0.044944
std,5.139097,0.384360,2.615373,12.379710,11.726573,0.498767,0.475673,0.455170,0.388981,0.207375
min,1.000000,0.000000,2.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.250000,0.000000,12.000000,8.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.780000,0.000000,12.000000,15.000000,35.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,11.250000,0.000000,15.000000,26.000000,44.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,44.500000,1.000000,18.000000,55.000000,64.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# X and y
# This data does not require much processing
X=df.drop('wage_per_hour',axis=1).values
y=df['wage_per_hour'].values

In [12]:
# scale it
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

# train test split
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)

(427, 9)
(107, 9)


In [13]:
# Define my model
n_cols=X_train.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [14]:
# compile our model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['root_mean_squared_error','mean_squared_error'])
model.fit(X_train,y_train,validation_split=0.2,epochs=20,verbose=False)

2025-10-01 14:52:58.954564: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50)             │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,065 (11.98 KB)

 Trainable params: 1,021 (3.99 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,044 (7.99 KB)

In [16]:
# predict the test data
y_pred=model.predict(X_test)
mean_squared_error(y_test,y_pred)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


24.795845585225006

In [17]:
## Compare this with a linear regression model
lin_reg=LinearRegression()
lin_reg.fit(X_train,y_train)
y_pred_lin=lin_reg.predict(X_test)
mean_squared_error(y_test,y_pred_lin)

21.24712426009875

In [18]:
## lets optimize the nn to see if it can improve
model=Sequential()
model.add(Dense(50,activation="relu"))
model.add(Dense(10,activation="relu"))
model.add(Dense(1,activation='relu'))# since this is a regression problem the class of the output is just one

In [19]:
# compile the model with SGD
learning_rates = [0.001,0.0001,0.1,0.003,0.3,0.0004,0.000001,0.0005]
for lr in learning_rates:
    model=model
    sgd=SGD(learning_rate=lr)
    model.compile(optimizer=sgd,loss='mean_squared_error')
    model.fit(X_train,y_train)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 90.8625   
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 76.5695  
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 214.3263 
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 106.6427 
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 106.6427 
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 106.6427 
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 106.6427  
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 106.6427  


In [20]:
## implement drop out
model=Sequential()
model.add(Dense(100,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(50,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation='relu'))

In [21]:
### stop overfitting by adding an early stopping criteria and drop out regularization
callbacks=EarlyStopping(patience=2)
sgd=SGD(learning_rate=0.001)
model.compile(optimizer=sgd,loss='mean_squared_error')
model.fit(X_train,y_train,epochs=30,validation_split=0.2,callbacks=[EarlyStopping(patience=2)])

Epoch 1/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 104.4245 - val_loss: 105.6713
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 100.5025 - val_loss: 101.3383
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 95.4385 - val_loss: 95.5968
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 87.4336 - val_loss: 85.6539
Epoch 5/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 74.2391 - val_loss: 66.3350
Epoch 6/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 52.4399 - val_loss: 41.4309
Epoch 7/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 37.9757 - val_loss: 29.0753
Epoch 8/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 34.4901 - val_loss: 27.2734
Epoch 9/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.3596 - val_loss: 25.5432
Epoch 10/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.0407 - val_loss: 23.2776
Epoch 11/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 31.9687 - val_loss: 22.8669
Epoch 12/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 